In [4]:
from ipywidgets import widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pandas as pd
import pickle

In [5]:
DIR_REPO = Path.cwd().parent.parent
DIR_DATA_PROCESSED = Path(DIR_REPO) / "data" / "processed"
DIR_MODELS = Path(DIR_REPO) / "models"

os.listdir(DIR_DATA_PROCESSED)
FILEPATH_PROCESSED = DIR_DATA_PROCESSED / "preprocessed_listings.csv"
df = pd.read_csv(FILEPATH_PROCESSED, index_col=0)

In [6]:
with open(DIR_MODELS / "simple_classifier.pkl", 'rb') as f:
    model = pickle.load(f)

In [7]:
def prepare_data(PT,Nbh,RT,Lat,Lon,Acm,Bath,Bedrms,Beds,Amnts):
    for i,p in enumerate(list(df['property_type'].unique())):
        if p == PT:
            PT = i+1
            break
    Nbh = {"Bronx": 1, "Queens": 2, "Staten Island": 3, "Brooklyn": 4, "Manhattan": 5}[Nbh]
    RT = {"Shared room": 1, "Private room": 2, "Entire home/apt": 3, "Hotel room": 4}[RT]
    
    TV,Internet,Air_conditioning,Kitchen,Heating,Wifi,Elevator,Breakfast = 0,0,0,0,0,0,0,0
    if "TV" in Amnts:
        TV=1
    if "Internet" in Amnts:
        Internet=1
    if "Air_conditioning" in Amnts:
        Air_conditioning=1
    if "Kitchen" in Amnts:
        Kitchen=1
    if "Heating" in Amnts:
        Heating=1
    if "Wifi" in Amnts:
        Wifi=1
    if "Elevator" in Amnts:
        Elevator=1
    if "Breakfast" in Amnts:
        Breakfast=1
    
    
    dic_tmp = { "neighbourhood": Nbh,
            "property_type":PT,
            "room_type": RT,
            "latitude": Lat,
            "longitude": Lon,
            "accommodates": Acm,
            "bathrooms": Bath,
            "bedrooms": Bedrms,
            "beds": Beds,
            "TV": TV,
            "Internet": Internet,
            "Air_conditioning":Air_conditioning,
            "Kitchen":Kitchen,
            "Heating":Heating,
            "Wifi":Wifi,
            "Elevator": Elevator,
            "Breakfast":Breakfast
    }
    pred_values = pd.Series(dic_tmp).values.reshape(1, -1)
    return pred_values

In [8]:
Property_Type = widgets.Dropdown(options =list(df['property_type'].unique()),description = 'Property type')

In [9]:
Neighbourhood = widgets.ToggleButtons(
    options=['Bronx', 'Queens', 'Staten Island', 'Brooklyn', 'Manhattan'],
    description='Neighbourhood:',
    disabled=False,
    button_style='info')

In [10]:
Room_Type = widgets.ToggleButtons(
    options=['Shared room', 'Private room', 'Entire home/apt', 'Hotel room'],
    description='Room type:',
    disabled=False,
    button_style='info')

In [11]:
Coords = widgets.HBox([widgets.Label(value="Latitude"), widgets.FloatText(),widgets.Label(value="Longitude"), widgets.FloatText()])

In [12]:
Acommodates = widgets.HBox([widgets.Label(value="Acommodates"), widgets.IntSlider(min=1,max=20)])

In [13]:
Bathrooms = widgets.HBox([widgets.Label(value="Bathrooms"), widgets.FloatSlider(min=0,max=10,step=0.5)])

In [14]:
Bedrooms = widgets.HBox([widgets.Label(value="Bedrooms"), widgets.IntSlider(min=1,max=15)])

In [15]:
Beds = widgets.HBox([widgets.Label(value="Beds"), widgets.IntSlider(min=1,max=30)])

In [16]:
Amenities = widgets.SelectMultiple(
    options=['TV', 'Internet', 'Air_conditioning','Kitchen','Heating','Wifi','Elevator','Breakfast'],
    rows=8,
    description='Amenities',
    disabled=False
)

In [20]:
btn = widgets.Button (description= "Predict")
clear_btn = widgets.Button (description= "Clear")
output = widgets.Output(layout={'border': '1px solid black'})
@output.capture()
def btn_eventhandler (obj):
    pred_values = prepare_data(Property_Type.value,
                               Neighbourhood.value,
                               Room_Type.value,
                               Coords.children[1].value,
                               Coords.children[3].value,
                               Acommodates.children[1].value,
                               Bathrooms.children[1].value,
                               Bedrooms.children[1].value,
                               Beds.children[1].value,
                               Amenities.value
                              )
    tmp = model.predict(pred_values)
    
    predicted_category = {0:'Low', 1:'Mid', 2:'High', 3:'Luxury'}[int(tmp[0])]
    with output:
        output.clear_output()
        print("Predicted Category:", predicted_category)
def clear_btn_event (obj):
    with output:
        output.clear_output()
        
btn.on_click(btn_eventhandler)
clear_btn.on_click(clear_btn_event)
display(output)

Output(layout=Layout(border='1px solid black'))

In [21]:
widgets.VBox([Property_Type, Neighbourhood,Room_Type,Coords,widgets.HBox([Acommodates, Bedrooms]),widgets.HBox([Bathrooms, Beds]),Amenities,widgets.HBox([btn, clear_btn]),output])